In [1]:
# !pip install nlp2go transformers tfkit -U

In [2]:
source_path = './tfkit-model-path'
target_path = "./bart-distractor-generation/"

In [3]:
import nlp2go
tfkit_model = nlp2go.Model(source_path)
tfkit_model.model.eval()

===model info===
model_config : facebook/bart-base
tags : ['seq2seq_0']
type : ['seq2seq']
maxlen : 1024
epoch : 8
loading saved model
Using device: cuda
finish loading


Model(
  (pretrained): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): BartEncoderLa

In [4]:
from transformers import AutoTokenizer, BartForConditionalGeneration
from transformers import pipeline
hf_model = BartForConditionalGeneration.from_pretrained('facebook/bart-base').to('cuda')
hf_model.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        

In [5]:
print(hf_model._modules.keys())
print(hf_model.model._modules.keys())

odict_keys(['model', 'lm_head'])
odict_keys(['shared', 'encoder', 'decoder'])


In [6]:
hf_model.lm_head = tfkit_model.model.model
hf_model.model = tfkit_model.model.pretrained

In [7]:
text = "shyness is the cause of much unhappiness for a great many people . shy people are anxious and self - conscious ; that is , they are over concerned with their own appearance and actions . worrisome thoughts are constantly occurring in their minds : what kind of impression am i making ? do they like me ? do i sound stupid ? am i wearing unattractive clothes ? it is obvious that such uncomfortable feelings must affect people unfavorably . adperson ' s self concept is reflected in the way he or she behaves and the way a person behaves affects other people ' s reactions . in general , the way people think about themselves has a deep effect on all areas of their lives . shy people , have low self - esteem , are likely to be passive and easily influenced by others . they need reassurance ( , ) that they are doing \" the right thing \" . shy people are very sensitive to criticism . it makes them feel inferior . they also find it difficult to be pleased by praises because they believe they are unworthy of praise . a shy person may respond to a praise with a statement like this one : \" you \' re just saying that to make me feel good . i know it ' s not true . \" it is clear that , while self - awareness is a healthy quality , overdoing it is harmful . can shyness be completely got rid of , or at least reduced ? fortunately , people can overcome shyness with determination . it is important for people to accept their weaknesses as well as their strengths , for example , not fair for them to label themselves inferior because they have to be realistic . living on the impossible leads to absence of inferiority . each one of us has his or her own characteristics . we are interested in our own personal ways . the better we understand ourselves . the easier it becomes to live up to our chances for a rich and fulfilling life . </s> according to the wirter , self - awareness is </s> a good characteristic"

In [8]:
inputs = tfkit_model.model.tokenizer(text, max_length=1024, return_tensors='pt').to('cuda')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [9]:
hf_model.config.tie_word_embeddings = False
hf_model.config.tie_encoder_decoder = False

In [10]:
input_ids = inputs['input_ids'].to('cuda')

In [11]:
generations = hf_model.generate(input_ids)
tfkit_model.model.tokenizer.decode(generations[0])

'</s>a good quality</s>'

In [12]:
hf_model.save_pretrained(target_path)
tfkit_model.model.tokenizer.save_pretrained(target_path)

('./bart-distractor-generation/tokenizer_config.json',
 './bart-distractor-generation/special_tokens_map.json',
 './bart-distractor-generation/vocab.json',
 './bart-distractor-generation/merges.txt',
 './bart-distractor-generation/added_tokens.json')

In [13]:
hf_model,info = BartForConditionalGeneration.from_pretrained(target_path, output_loading_info=True)
print(info)
hf_model.to('cuda')
generations = hf_model.generate(inputs['input_ids'])
tfkit_model.model.tokenizer.decode(generations[0])

{'missing_keys': [], 'unexpected_keys': [], 'error_msgs': []}


'</s>a good quality</s>'